In [1]:
import sys
import os

# Add the directory containing your modules to the system path
sys.path.insert(0, os.path.abspath('/projects/e32704/andy/decisionmaking/gridworld-decisionmaking/Simulation 2 Pseudo-Terrestrial/planning'))

In [78]:
import pickle as pkl
import pandas as pd
import numpy as np

from game import Game
from simulator import Knowledge
from mcts import SearchParams, UnitTestMCTS
from experiment import ExperimentParams, Experiment
from entropy import UnitTestENTROPY
from utils import UnitTestUTILS, Random
from coord import COORD, UnitTestCOORD, COMPASS, ManhattanDistance, Compass, LineCoordinates
from grid import Grid
from entropy import Entropy
from astar import AStar
from math import floor
from time import time

In [3]:
init_vars_path = "/projects/e32704/andy/decisionmaking/gridworld-results/initial_test/init_vars.pkl"

In [5]:
with open(init_vars_path, 'rb') as f:
    init_vars = pkl.load(f)

In [9]:
type(init_vars)

list

In [10]:
len(init_vars)

2

In [12]:
type(init_vars[0])

list

In [13]:
type(init_vars[1])

list

In [14]:
len(init_vars[0])

20

In [15]:
len(init_vars[1])

20

In [17]:
type(init_vars[0][0])

list

In [19]:
len(init_vars[0][0])

10

In [20]:
init_vars[0][0][0]

[]

In [21]:
init_vars[0][0][1]

In [22]:
init_vars[0][0][2]

In [28]:
for i, val in enumerate(init_vars[0][0]):
    # print(type(val))
    # print(len(val))
    if len(val) > 0:
        # print(val[1])
        print(len(val) == len(set(val)))

True
False
False
False
False
False
False
False
False


My guess is that `init_vars[0][0]` specifies the occlusions for an environment. Specifically, I believe that the second index specifies occlusions - the first index is likely some other high-level factor, since there are only two items at the first index.

### Step through construction logic

In [6]:
XSize = 15
YSize = 15
AgentHome = COORD(7, 0)
GoalPos = COORD(floor(XSize/2), YSize-1)
visualRange = None

In [58]:
def VisualCone(self, coord, visualRange):
    temp = []
    for dec in range(visualRange):
        temp.append(coord[dec, dec:(2 * visualRange - dec) + 1])
    return [y for x in temp for y in x]  # if self.Inside(y)]

In [65]:
def VisualArea(self, coord, observationDirection, visualRange, pureVision=False):
    RadiusCoordinates = []
    for x in range(coord.X - visualRange, coord.X + visualRange + 1):
        for y in range(coord.Y - visualRange, coord.Y + visualRange + 1):
            if (ManhattanDistance(COORD(x, y), coord) <= 2 * visualRange):
                RadiusCoordinates.append(COORD(x, y))

    RangeCoordinates = []
    RadiusCoordinates = np.flipud(
        np.reshape(np.array(RadiusCoordinates), (2 * visualRange + 1, 2 * visualRange + 1)).transpose())
    if observationDirection == COMPASS.NORTH:
        RangeCoordinates = self.VisualCone(RadiusCoordinates, visualRange)

    elif observationDirection == COMPASS.EAST:
        RadiusCoordinates = np.flipud(RadiusCoordinates.transpose())
        RangeCoordinates = self.VisualCone(RadiusCoordinates, visualRange)

    elif observationDirection == COMPASS.WEST:
        RadiusCoordinates = RadiusCoordinates.transpose()
        RangeCoordinates = self.VisualCone(RadiusCoordinates, visualRange)

    elif observationDirection == COMPASS.SOUTH:
        RadiusCoordinates = np.flipud(RadiusCoordinates)
        RangeCoordinates = self.VisualCone(RadiusCoordinates, visualRange)

    assert (RangeCoordinates)

    if pureVision:
        return RangeCoordinates

    for a in range(4):
        sidePos = coord + Compass[a]
        if sidePos not in RangeCoordinates:
            RangeCoordinates.append(sidePos)

    return RangeCoordinates

In [79]:
observationDirection = COMPASS.NORTH
RadiusCoordinates = []
for x in range(coord.X - visualRange, coord.X + visualRange + 1):
    for y in range(coord.Y - visualRange, coord.Y + visualRange + 1):
        print(x, y)
        if (ManhattanDistance(COORD(x, y), coord) <= 2 * visualRange):
            RadiusCoordinates.append(COORD(x, y))

# RangeCoordinates = []
# RadiusCoordinates = np.flipud(
#     np.reshape(np.array(RadiusCoordinates), (2 * visualRange + 1, 2 * visualRange + 1)).transpose())
# if observationDirection == COMPASS.NORTH:
#     RangeCoordinates = self.VisualCone(RadiusCoordinates, visualRange)

# elif observationDirection == COMPASS.EAST:
#     RadiusCoordinates = np.flipud(RadiusCoordinates.transpose())
#     RangeCoordinates = self.VisualCone(RadiusCoordinates, visualRange)

# elif observationDirection == COMPASS.WEST:
#     RadiusCoordinates = RadiusCoordinates.transpose()
#     RangeCoordinates = self.VisualCone(RadiusCoordinates, visualRange)

# elif observationDirection == COMPASS.SOUTH:
#     RadiusCoordinates = np.flipud(RadiusCoordinates)
#     RangeCoordinates = self.VisualCone(RadiusCoordinates, visualRange)

# assert (RangeCoordinates)

# if pureVision:
#     return RangeCoordinates

# for a in range(4):
#     sidePos = coord + Compass[a]
#     if sidePos not in RangeCoordinates:
#         RangeCoordinates.append(sidePos)

# return RangeCoordinates

2 -5
2 -4
2 -3
2 -2
2 -1
2 0
2 1
2 2
2 3
2 4
2 5
3 -5
3 -4
3 -3
3 -2
3 -1
3 0
3 1
3 2
3 3
3 4
3 5
4 -5
4 -4
4 -3
4 -2
4 -1
4 0
4 1
4 2
4 3
4 4
4 5
5 -5
5 -4
5 -3
5 -2
5 -1
5 0
5 1
5 2
5 3
5 4
5 5
6 -5
6 -4
6 -3
6 -2
6 -1
6 0
6 1
6 2
6 3
6 4
6 5
7 -5
7 -4
7 -3
7 -2
7 -1
7 0
7 1
7 2
7 3
7 4
7 5
8 -5
8 -4
8 -3
8 -2
8 -1
8 0
8 1
8 2
8 3
8 4
8 5
9 -5
9 -4
9 -3
9 -2
9 -1
9 0
9 1
9 2
9 3
9 4
9 5
10 -5
10 -4
10 -3
10 -2
10 -1
10 0
10 1
10 2
10 3
10 4
10 5
11 -5
11 -4
11 -3
11 -2
11 -1
11 0
11 1
11 2
11 3
11 4
11 5
12 -5
12 -4
12 -3
12 -2
12 -1
12 0
12 1
12 2
12 3
12 4
12 5


In [60]:
visualRange = 5

In [62]:
coord = AgentHome

In [63]:
temp = []
for dec in range(visualRange):
    temp.append(coord[dec, dec:(2 * visualRange - dec) + 1])
return [y for x in temp for y in x]  # if self.Inside(y)]

TypeError: 'COORD' object is not subscriptable

In [43]:
occlusions = Grid(XSize, YSize).CreateOcclusions(numOcclusions, AgentHome, GoalPos)

In [7]:
def GetEntropy(occlusions):
    I = np.zeros((XSize, YSize))
    for coord in occlusions:
        I[coord.X, coord.Y] = 1
    entropy = Entropy(I)
    outputMatrix = entropy.MovingWindowFilter(entropy.MovingAverage, 1)
    filteredMatrices = [outputMatrix]
    profile = entropy.Profile(filteredMatrices)

    return profile

In [11]:
def GetPath(occlusions):
    ASTAR = AStar(XSize, YSize, occlusions)
    ASTAR.InitializeGrid(AgentHome, GoalPos)
    path = ASTAR.Solve()

    return path

In [12]:
occlusionList = [[[] for occlusionInd in ExperimentParams.EntropyLevels]
                        for simulationInd in range(ExperimentParams.NumRuns)]
predatorList = [[[] for occlusionInd in ExperimentParams.EntropyLevels]
                 for simulationInd in range(ExperimentParams.NumRuns)]
for simulationInd in range(ExperimentParams.NumRuns):
    for i, entropyValue in enumerate(ExperimentParams.EntropyLevels):
        while True and entropyValue < .8:
            numOcclusions = 0
            occlusions = Grid(XSize, YSize).CreateOcclusions(numOcclusions, AgentHome, GoalPos)
            while GetEntropy(occlusions)[0] < entropyValue:
                #entropyMatrix = GetEntropy(occlusions)[0]
                #if entropyMatrix >= entropyValue:
                #    break
                occlusions = Grid(XSize, YSize).CreateOcclusions(numOcclusions, AgentHome, GoalPos)
                numOcclusions += 1
            path = GetPath(occlusions)
            if path:
                occlusionList[simulationInd][i] = occlusions
                break

        while True and entropyValue >= .8:
            numOcclusions = 20
            occlusions = Grid(XSize, YSize).CreateRandomOcclusions(numOcclusions, AgentHome, GoalPos)
            while GetEntropy(occlusions)[0] < entropyValue:
                occlusions = Grid(XSize, YSize).CreateOcclusions(numOcclusions, AgentHome, GoalPos)
                numOcclusions += 1

            path = GetPath(occlusions)
            if path:
                occlusionList[simulationInd][i] = occlusions
                break

        allPredatorLocations = Grid(XSize, YSize).CreatePredatorLocations(ExperimentParams.SpawnArea, AgentHome,
                                                                  GoalPos,occlusions)

        temp_predator = [allPredatorLocations[Random(0, len(allPredatorLocations))]
                             for p in range(ExperimentParams.NumPredators)]
        predatorList[simulationInd][i] = temp_predator

### Notes

Each of `occlusionList` and `predatorList` is a list, and specifically a 2d list where the outer list represents the number of simulation runs (`ExperimentParams.NumRuns`, 20 runs) and the inner list represents different entropy levels (`ExperimentParams.EntropyLevels`, 10 levels).

A unique set of occlusions is generated for 



* index 0 corresponds with two lists - 0 is the occlusionList, and 1 is the predatorList
* 